In [1]:
#import necessary stuff
import requests
import json
import networkx

In [2]:
# define header information
headers = {
    'Ocp-Apim-Subscription-Key': '078f222c1e244f929fe773f0c38604ad',
}

In [3]:
# define research fields of interest for us
fields_of_studies = ['Machine learning']
top_5_conf_series_per_field = {}

In [4]:
# crawl top 10 cited conference series for each field of study
for field_of_study in fields_of_studies:
    top_5_conf_series_per_field[field_of_study] = {}
    params = {
        'expr': "Composite(F.FN=='Machine learning')",
        'model': 'latest',
        'attributes': 'Id,CN',
        'count': '5',
        'offset': '0',
        'orderby': 'CC:desc'
    }

    params_url = 'expr={}&model={}&count={}&offset={}&attributes={}&orderby={}'.format(params['expr'], params['model'], params['count'], params['offset'], params['attributes'], params['orderby'])

    base_url = 'https://api.labs.cognitive.microsoft.com'
    query = '/academic/v1.0/evaluate?{}'.format(params_url)

    session = requests.Session()
    r = session.get(base_url + query, stream=False, headers=headers)
    response_body = r.content.decode('utf-8')
    
    top_5_results = dict(json.loads(response_body))['entities']
    top_5_conference_series_ids = []
    
    for result in top_5_results:
        top_5_conference_series_ids.append(result['Id'])
        
    for conf_id in top_5_conference_series_ids:
        top_5_conf_series_per_field[field_of_study][conf_id] = {}

In [5]:
# define years of interest 
years = [2013,2014,2015,2016]
max_papers_per_year = 500

In [6]:
# crawl top-500 cited papers for each conference id between 2013 and 2016 
for field_of_study in fields_of_studies:
    for conference_series_id in top_5_conf_series_per_field[field_of_study]:
        for year in years:
            top_5_conf_series_per_field[field_of_study][conference_series_id][year] = []
            
            params = {
                'expr': "AND(Composite(C.CId={}), Y={})".format(conference_series_id, year),
                'model': 'latest',
                'attributes': 'Id,CC,RId,AA.AuId',
                'count': max_papers_per_year,
                'offset': '0',
                'orderby': 'CC:desc'
            }

            params_url = 'expr={}&model={}&count={}&offset={}&attributes={}&orderby={}'.format(params['expr'], params['model'], params['count'], params['offset'], params['attributes'], params['orderby'])

            base_url = 'https://api.labs.cognitive.microsoft.com'
            query = '/academic/v1.0/evaluate?{}'.format(params_url)

            session = requests.Session()
            r = session.get(base_url + query, stream=False, headers=headers)
            response_body = r.content.decode('utf-8')

            top_500_results = dict(json.loads(response_body))['entities']
            for result in top_500_results:
                reference_ids = []
                author_ids = []
                
                if 'RId' in result:
                    reference_ids = result['RId']
                    
                if 'AA' in result:
                    author_ids = [author['AuId'] for author in result['AA']]
                
                paper_info = {
                    'Id': result['Id'],
                    'CC': result['CC'],
                    'RId': reference_ids,
                    'authors': author_ids
                }
                top_5_conf_series_per_field[field_of_study][conference_series_id][year].append(paper_info)

In [7]:
import pickle

coauthor_crawled_data_file_path = 'data/coauthor/coauthor_json.p'
with open(coauthor_crawled_data_file_path, 'wb') as pickle_file:
    pickle.dump(top_5_conf_series_per_field, pickle_file)